In [82]:
import pandas as pd
import math

In [103]:
# n day retention


# WIN
#users_sleepy = pd.read_csv(r'C:\Users\Incognitus\Downloads\entries.csv', sep=';') 
# MAC
users_sleepy = pd.read_csv(r'/Users/vladislavlipkin/Downloads/entries.csv', sep=';') 

In [104]:
users_sleepy.head(5)

,user,dt,Unnamed: 2
0,0,12.11.2023,NaN
1,0,13.11.2023,NaN
2,0,14.11.2023,NaN
3,0,16.11.2023,NaN
4,0,17.11.2023,NaN


In [105]:
# Берем нужные столбцы из исходного датафрейма
users_filter = users_sleepy.loc[:, ['user', 'dt']]

# Преобразуем столбец dt в формат datetime
users_filter['dt'] = pd.to_datetime(users_filter['dt'], format='%d.%m.%Y')

# Определяем дату первого визита (регистрации) каждого пользователя
users_filter['dt_reg'] = users_filter.groupby('user')['dt'].transform('min')

# Считаем разницу в днях между визитом и регистрацией
users_filter['diff'] = (users_filter['dt'] - users_filter['dt_reg']).dt.days

# Считаем общее количество уникальных пользователей
all_users = users_filter['user'].nunique()

# Вычисляем retention: доля уникальных пользователей в каждом diff
users_filter['retention'] = users_filter.groupby('diff')['user'].transform('nunique') / all_users * 100

# Создаем сводную таблицу retention по дням
pivot = users_filter.pivot_table(
    index='diff',
    values='retention',
    aggfunc='mean'
).reset_index()

# Создаем полный диапазон diff от min до max (чтобы не было пропусков)
full_range = pd.DataFrame({'diff': range(pivot['diff'].min(), pivot['diff'].max() + 1)})

# Объединяем полный диапазон с pivot (left join)
pivot_full = full_range.merge(pivot, on='diff', how='left')

# Просматриваем первые строки итоговой таблицы
pivot_full.head(20)

In [106]:
users_filter['dt'] = pd.to_datetime(users_filter['dt'], format='%d.%m.%Y')

In [88]:
users_filter.head(10)

,user,dt
0,0,2023-11-12
1,0,2023-11-13
2,0,2023-11-14
3,0,2023-11-16
4,0,2023-11-17
5,0,2023-11-18
6,0,2023-11-19
7,0,2023-11-21
8,0,2023-11-22
9,0,2023-11-23


In [112]:
users_filter['dt_reg'] = users_filter.groupby('user')['dt'].transform('min')
users_filter['diff'] = (users_filter['dt'] - users_filter['dt_reg']).dt.days
users_filter.head(10)

,user,dt,dt_reg,diff
0,0,2023-11-12,2023-11-12,0
1,0,2023-11-13,2023-11-12,1
2,0,2023-11-14,2023-11-12,2
3,0,2023-11-16,2023-11-12,4
4,0,2023-11-17,2023-11-12,5
5,0,2023-11-18,2023-11-12,6
6,0,2023-11-19,2023-11-12,7
7,0,2023-11-21,2023-11-12,9
8,0,2023-11-22,2023-11-12,10
9,0,2023-11-23,2023-11-12,11


In [102]:
all_users = users_filter['user'].nunique()
all_users

2000

In [120]:
users_filter['retention'] = users_filter.groupby('diff')['user'].transform('nunique') / all_users * 100
users_filter.head(20)

,user,dt,dt_reg,diff,retention
0,0,2023-11-12,2023-11-12,0,100.00
1,0,2023-11-13,2023-11-12,1,51.95
2,0,2023-11-14,2023-11-12,2,52.60
3,0,2023-11-16,2023-11-12,4,51.20
4,0,2023-11-17,2023-11-12,5,48.40
5,0,2023-11-18,2023-11-12,6,49.20
6,0,2023-11-19,2023-11-12,7,46.35
7,0,2023-11-21,2023-11-12,9,47.90
8,0,2023-11-22,2023-11-12,10,47.05
9,0,2023-11-23,2023-11-12,11,44.85


In [124]:
# создаем pivot
pivot = users_filter.pivot_table(
    index='diff',
    values='retention',
    aggfunc='mean'
).reset_index()

# создаем полный диапазон diff от min до max
full_range = pd.DataFrame({'diff': range(pivot['diff'].min(), pivot['diff'].max() + 1)})

# объединяем с pivot (левый join)
pivot_full = full_range.merge(pivot, on='diff', how='left')

pivot_full.head(20)

,diff,retention
0,0,100.00
1,1,51.95
2,2,52.60
3,3,52.75
4,4,51.20
5,5,48.40
6,6,49.20
7,7,46.35
8,8,47.70
9,9,47.90


In [107]:
# all_users = count_day_use_app['count_day_use'].count()
# day_x = count_day_use_app.query('count_day_use > 14').count()
# day_14 = count_day_use_app.query('count_day_use == 14').count()

# retention_day_x = day_x / all_users * 100
# retention_day_14 = day_14 / all_users * 100

# print(math.ceil(abs(retention_day_14 - retention_day_x)))
# print()
# print(count_day_use_app['count_day_use'].count())
# print(count_day_use_app.query('count_day_use < 15').count())
# print(count_day_use_app.query('count_day_use > 14').count())

In [62]:
# должно быть 18

-56
